# 資料處理

In [1]:
import pandas as pd

## 資料讀取
### 讀取方式

In [ ]:
txt_data = pd.read_csv("data/demo1.txt", sep='\t')
csv_data = pd.read_csv("data/demo2.csv")

In [ ]:
import pandas as pd
txt_data = pd.read_csv("data/demo1.txt", sep='\t')
csv_data = pd.read_csv("data/demo2.csv")

## 資料清洗
### 變數結構

In [ ]:
# 資料總數
txt_data.size

# 資料維度資訊
txt_data.ndim # 維度數目
txt_data.shape # 維度長度 

# 變數型態
txt_data.dtypes

# 資料檢視
txt_data.describe
txt_data.info

### 遺失值處理


In [ ]:
txt_data_missing = pd.concat([txt_data, pd.DataFrame({'ID': [99], 'ListID':['List5'], 'Hearing':[pd.NA]})])

In [ ]:
# 尋找遺失值
txt_data_missing.isnull()

# 遺失值數量統計
txt_data_missing.isnull().sum()

In [ ]:
# 遺失值填補為 0
txt_data_clean = txt_data_missing.fillna(0)

# 遺失值填補平均值
hearing_maen = txt_data_missing['Hearing'].mean()
txt_data_clean = txt_data_missing.fillna(hearing_maen)

# 刪除 NA 所在之觀察值
txt_data_clean = txt_data_missing.dropna()

### 重複值處理

In [ ]:
txt_data1 = txt_data.drop_duplicates()

### 異常值檢測

In [ ]:
#####
# 標準差
#####
txt_data.std()

#####
# 四分位距(IQR)
#####
# 自定義IQR程式碼
def fetch_IQR(data:pd.DataFrame, col:str):
    return data[col].quantile(.75)-data[col].quantile(.25)

# txt_data 中 Hearing 的 IQR
fetch_IQR(data = txt_data, col='Hearing')

## 資料轉換

### 資料正規化/標準化

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
txt_data_trans = txt_data.copy()

# 資料正規化
x_scale_norm = txt_data_trans.loc[:,['Hearing']]
x_scale_norm = MinMaxScaler().fit_transform(x_scale_norm)

# 資料標準化
x_scale_std = txt_data_trans.loc[:,['Hearing']]
x_scale_std = StandardScaler().fit_transform(x_scale_std)

### 虛擬變數

In [ ]:
txt_data_dummy = txt_data.copy()

# 將 ListID 轉化成虛擬變數
pd.get_dummies(txt_data_dummy, prefix=['ListID'])

# 資料分析
載入 LARS 資料

In [5]:
df_lars_data = pd.read_csv("data/LARS_flu.csv")
df_lars_data = df_lars_data.dropna()
df_lars_data

,Year-Week of Specimen Received,Adenovirus,Parainfluenza,HSV,CMV,RSV,Influenza,SARS-CoV-2,HMPV,% Positive
0,202240,7.0,37.0,5.0,1.0,12.0,4.0,1.0,0.0,18.9
1,202241,1.0,50.0,2.0,3.0,15.0,6.0,5.0,0.0,30.4
2,202242,1.0,33.0,1.0,1.0,8.0,6.0,3.0,0.0,29.1
3,202243,2.0,73.0,29.0,0.0,18.0,5.0,4.0,0.0,21.2
4,202244,2.0,40.0,7.0,0.0,16.0,4.0,3.0,0.0,24.7
...,...,...,...,...,...,...,...,...,...,...
89,202425,21.0,1.0,8.0,0.0,0.0,35.0,15.0,0.0,27.2
90,202426,8.0,1.0,3.0,1.0,3.0,19.0,14.0,2.0,17.4
91,202427,9.0,4.0,1.0,0.0,4.0,25.0,7.0,0.0,30.1
92,202428,5.0,1.0,1.0,1.0,6.0,36.0,7.0,0.0,20.6


## 描述性分析

In [9]:
# 建立描述性分析所需的資料
def fetch_data_basic_info(data:pd.DataFrame, colname:str) -> dict:
    basic_info = {
        'Mean' : data[colname].mean(),
        'StdDev' : data[colname].std(),
        'Min' : data[colname].min(),
        'Q1' : data[colname].quantile(0.25),
        'Median' : data[colname].median(),
        'Q3' : data[colname].quantile(0.75),
        'Max' : data[colname].max()
    }
    return basic_info

# 呈現結果
print(fetch_data_basic_info(data=df_lars_data, colname='Parainfluenza'))

In [10]:
print(fetch_data_basic_info(data=df_lars_data, colname='Parainfluenza'))

{'Mean': 8.797872340425531, 'StdDev': 11.237016199098258, 'Min': 0.0, 'Q1': 2.0, 'Median': 5.5, 'Q3': 2.0, 'Max': 0.0}


## 樞紐分析表

In [18]:
df_piv_lars_data = df_lars_data.melt(id_vars='Year-Week of Specimen Received')
df_piv_lars_data

,Year-Week of Specimen Received,variable,value
0,202240,Adenovirus,7.0
1,202241,Adenovirus,1.0
2,202242,Adenovirus,1.0
3,202243,Adenovirus,2.0
4,202244,Adenovirus,2.0
...,...,...,...
841,202425,% Positive,27.2
842,202426,% Positive,17.4
843,202427,% Positive,30.1
844,202428,% Positive,20.6


In [25]:
piv_lars = pd.pivot_table(
    data = df_piv_lars_data, 
    index = 'Year-Week of Specimen Received',
    columns = 'variable',
    aggfunc = 'mean'
)
piv_lars

value                             \
variable                       % Positive Adenovirus  CMV HMPV   HSV   
Year-Week of Specimen Received                                         
202240                               18.9        7.0  1.0  0.0   5.0   
202241                               30.4        1.0  3.0  0.0   2.0   
202242                               29.1        1.0  1.0  0.0   1.0   
202243                               21.2        2.0  0.0  0.0  29.0   
202244                               24.7        2.0  0.0  0.0   7.0   
...                                   ...        ...  ...  ...   ...   
202425                               27.2       21.0  0.0  0.0   8.0   
202426                               17.4        8.0  1.0  2.0   3.0   
202427                               30.1        9.0  0.0  0.0   1.0   
202428                               20.6        5.0  1.0  0.0   1.0   
202429                               20.8        2.0  0.0  0.0   2.0   

                                                                         
variable                       Influenza Parainfluenza   RSV SARS-CoV-2  
Year-Week of Specimen Received                                           
202240                               4.0          37.0  12.0        1.0  
202241                               6.0          50.0  15.0        5.0  
202242                               6.0          33.0   8.0        3.0  
202243                               5.0          73.0  18.0        4.0  
202244                               4.0          40.0  16.0        3.0  
...                                  ...           ...   ...        ...  
202425                              35.0           1.0   0.0       15.0  
202426                              19.0           1.0   3.0       14.0  
202427                              25.0           4.0   4.0        7.0  
202428                              36.0           1.0   6.0        7.0  
202429                              16.0           2.0   0.0        8.0  

[94 rows x 9 columns]